In [1]:
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *

import sys

In [8]:
class interface(QWidget):
    """
    This is my widget.
    """
    def __init__(self):
        super().__init__()
        
        #layout principal
        self.global_layout = QHBoxLayout(self)
        
        self.tab_widget = QTabWidget(self)
        self.tab1 = child_widget_1(self.tab_widget)
        self.tab_widget.addTab(self.tab1,"Live Mode")
        
        self.tab2 = child_widget_2(self.tab_widget)
        self.tab_widget.addTab(self.tab2,"Calibration Mode")
        
        shadow1 = QGraphicsDropShadowEffect(xOffset=3,yOffset=3,blurRadius=5,color=Qt.black)
        self.tab_widget.setGraphicsEffect(shadow1)
        
        self.global_layout.addWidget(self.tab_widget)
        
################################################################### 

class child_widget_1(QWidget):
    
    def __init__(self, parent=None):
        super().__init__(parent)
        mainLayout=QBoxLayout(self)
        
    def laser_widget():
        return 0
###################################################################        

class child_widget_2(QWidget):
    def __init__(self, parent=None):
        super().__init__(parent=parent)
        
        self.global_layout = QHBoxLayout(self)
        
        
###################################################################

qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
    
if __name__ == "__main__": 
    #start the widget
    ui = interface()
    #show the widget
    ui.show()
    #start the event loop
    qapp.exec_()

TypeError: QBoxLayout(QBoxLayout.Direction, parent: QWidget = None): argument 1 has unexpected type 'child_widget_1'

# Camera

In [3]:
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from pylablib.devices import uc480 #uc480 dll's needed
import cv2
import sys
import numpy as np
from live import LiveFeed

class camera_widget(QWidget):
    
    def __init__(self):
        super().__init__()
    
        self.setWindowTitle("Camera")
        self.disply_width = 640
        self.display_height = 480
        # create the label that holds the image
        self.image_label = QLabel(self)
        self.image_label.resize(self.disply_width, self.display_height)
        # create a text label
        self.textLabel = QLabel('Camera')

        # create a vertical box layout and add the two labels
        vbox = QVBoxLayout()
        vbox.addWidget(self.image_label)
        vbox.addWidget(self.textLabel)
        # set the vbox layout as the widgets layout
        self.setLayout(vbox)

        # create the video capture thread
        self.thread = LiveFeed()
        # connect its signal to the update_image slot
        self.thread.change_pixmap_signal.connect(self.update_image)
        # start the thread
        self.thread.start()
        

    def closeEvent(self, event):
        self.thread.stop()
        event.accept()

    #@pyqtSlot(np.ndarray)
    def update_image(self, frame):
        """Updates the image_label with a new opencv image"""
        qt_img = self.convert_cv_qt(frame)
        painter = QPainter(qt_img)
        pen = QPen()
        #pen.setWidth(40)
        pen.setColor(QColor('red'))
        painter.setPen(pen)
        painter.drawLine(270, 0, 270, 600)
        painter.drawLine(0, 247, 600, 247)
        painter.end()
        self.image_label.setPixmap(qt_img)
    
    def convert_cv_qt(self, frame):
        """Convert from an opencv image to QPixmap"""
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_image.shape
        bytes_per_line = ch * w
        convert_to_Qt_format = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        p = convert_to_Qt_format.scaled(self.disply_width, self.display_height, Qt.KeepAspectRatio)
        return QPixmap.fromImage(p)
    
qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
    
if __name__ == "__main__": 
    #start the widget
    ui = camera_widget()
    #show the widget
    ui.show()
    #start the events loop
    qapp.exec_()

# Sidebar Widget

In [4]:
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
import time

import sys

class sidebar_widget(QWidget):
    
    def __init__(self, parent=None):
        super().__init__(parent)
        
        sizePolicy=QSizePolicy(QSizePolicy.Expanding,QSizePolicy.Minimum)
        
        self.global_layout = QVBoxLayout(self)
        
        self.live_button=QToolButton(self)
        self.live_button.setText('Live Mode')
        self.live_button.setSizePolicy(sizePolicy)
        self.global_layout.addWidget(self.live_button)
        
        self.calibration_button=QToolButton(self)
        self.calibration_button.setSizePolicy(sizePolicy)
        self.calibration_button.setText('Calibration Mode')
        self.global_layout.addWidget(self.calibration_button)
    
        self.classification_button=QToolButton(self)
        self.classification_button.setSizePolicy(sizePolicy)
        self.classification_button.setText('Classification Mode')
        self.global_layout.addWidget(self.classification_button)
        
        spacer=QSpacerItem(40,40,QSizePolicy.Minimum,QSizePolicy.Expanding)
        self.global_layout.addItem(spacer)
        
        self.settings_button=QToolButton(self)
        self.settings_button.setSizePolicy(sizePolicy)
        self.settings_button.setText('Settings')
        self.global_layout.addWidget(self.settings_button)
        
        self.setFixedWidth(100)
        
qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
    
if __name__ == "__main__": 
    #start the widget
    ui = sidebar_widget()
    #show the widget
    ui.show()
    #start the events loop
    qapp.exec_()

# Laser

In [2]:
from PyQt5.QtCore import *
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
import time
import queue
from laser_utils.send_requests import req_Worker
from laser_utils.event_handler import Worker
from laser_utils.laser_Copy1 import mwLaser
#import breeze_resources

import sys

class laser_widget(QWidget):
    """
    This is my widget.
    """
    def __init__(self, parent=None):
        
        super().__init__()
        
        self.global_layout = QGridLayout(self)
        
        self.laser=mwLaser() ##laser driver
        
        self.q = queue.Queue(maxsize=10) ## Event queue
        
        #######################################################laser box
        container_laser=QGroupBox("Laser")
        vbox_laser= QVBoxLayout(container_laser)
        
        containersimple=QGroupBox()
        hboxsimple=QHBoxLayout(containersimple)
        self.current_display=QLCDNumber(self)
        self.current_display.setGeometry(QRect())
        hboxsimple.addWidget(self.current_display)
        self.labelmAl=QLabel(self)
        self.labelmAl.setText('mA')
        hboxsimple.addWidget(self.labelmAl)
        vbox_laser.addWidget(containersimple)
        
        containersimple1=QGroupBox()
        hboxsimple1=QHBoxLayout(containersimple1)
        self.power_display=QLCDNumber(self)
        self.power_display.setGeometry(QRect())
        hboxsimple1.addWidget(self.power_display)
        self.labelmW=QLabel(self)
        self.labelmW.setText('mW')
        hboxsimple1.addWidget(self.labelmW)
        hboxsimple1.addWidget(containersimple1)
        vbox_laser.addWidget(containersimple1)
        
        ################################################TEC box
        container_TEC=QGroupBox("TEC")
        vbox_TEC= QVBoxLayout(container_TEC)
        containersimple2=QGroupBox()
        
        hboxsimple2=QHBoxLayout(containersimple2)
        self.temp_display=QLCDNumber(self)
        self.temp_display.setGeometry(QRect())
        hboxsimple2.addWidget(self.temp_display)
        self.labelC=QLabel(self)
        self.labelC.setText('ºC')
        hboxsimple2.addWidget(self.labelC)
        vbox_TEC.addWidget(containersimple2)
        
        containersimple3=QGroupBox()
        hboxsimple3=QHBoxLayout(containersimple3)
        self.currTEC_display=QLCDNumber(self)
        self.currTEC_display.setGeometry(QRect())
        hboxsimple3.addWidget(self.currTEC_display)
        self.labelmATEC=QLabel(self)
        self.labelmATEC.setText('mA')
        hboxsimple3.addWidget(self.labelmATEC)
        vbox_TEC.addWidget(containersimple3)
        
        ###############################################current control
        container_input=QGroupBox("Laser Setpoint")
        hbox_set= QHBoxLayout(container_input)
        
        self.labelcur=QLabel(self)
        self.labelcur.setText('Current setpoint:')
        hbox_set.addWidget(self.labelcur)
        
        self.currentv=QLineEdit(self)
        self.currentv.setText('50')
        self.currentv.returnPressed.connect(self.current_changed)
        hbox_set.addWidget(self.currentv)
        
        self.labelmA=QLabel(self)
        self.labelmA.setText('mA')
        hbox_set.addWidget(self.labelmA)
        
        ##################################################temp control
        container_inputt=QGroupBox("TEC Setpoint")
        hbox_set1= QHBoxLayout(container_inputt)
        
        self.labeltemps=QLabel(self)
        self.labeltemps.setText('Temperature setpoint:')
        hbox_set1.addWidget(self.labeltemps)
        
        self.temperaturev=QLineEdit(self)
        self.temperaturev.setText('25')
        self.temperaturev.returnPressed.connect(self.temp_changed)
        hbox_set1.addWidget(self.temperaturev)
        
        self.labelC=QLabel(self)
        self.labelC.setText('ºC')
        hbox_set1.addWidget(self.labelC)
        
        ########################################################buttons
        container_button=QGroupBox()
        hbox_set2= QHBoxLayout(container_button)
        
        self.turn_on_button=QToolButton(self)
        self.turn_on_button.setText('Start Laser')
        self.turn_on_button.clicked.connect(self.laser_clicked)
        #self.turn_on_button.clicked.connect(self.button_state)
        
        container_button2=QGroupBox()
        hbox_set3= QHBoxLayout(container_button2)
        
        self.shutdown_button=QToolButton(self)
        self.shutdown_button.setText('Shutdown')
        self.shutdown_button.clicked.connect(self.shutdown)
        #self.shutdown_button.clicked.connect(self.button_state)
        
        hbox_set2.addWidget(self.turn_on_button)
        hbox_set3.addWidget(self.shutdown_button)

        ######################################################global
        
        self.global_layout.addWidget(container_laser,0,0)
        self.global_layout.addWidget(container_TEC,0,1)
        self.global_layout.addWidget(container_input,1,0)
        self.global_layout.addWidget(container_inputt,1,1)
        self.global_layout.addWidget(container_button,2,1)
        self.global_layout.addWidget(container_button2,2,0)
        
        #self.setFixedWidth(100)
        #self.setFixedHeight(200)
    ###########################################################functions
    #### initial state   
        self.request_thread()
        self.thread()
        #self.button_state()
    #### initiate thread and change status
    
    def thread(self):
        self.thread=QThread()
        self.worker=Worker(self.q,self.laser)
        self.worker.moveToThread(self.thread)
        self.thread.started.connect(self.worker.update)
        self.worker.finished.connect(self.thread.quit)
        self.worker.finished.connect(self.worker.deleteLater)
        self.thread.finished.connect(self.thread.deleteLater)
        self.worker.data.connect(self.update_status)
        self.thread.start()
    
    def request_thread(self):
        self.reqthread=QThread()
        self.reqworker=req_Worker(self.q)
        self.reqworker.moveToThread(self.reqthread)
        self.reqthread.started.connect(self.reqworker.ask)
        self.reqworker.finished.connect(self.reqthread.quit)
        self.reqworker.finished.connect(self.reqworker.deleteLater)
        self.reqthread.finished.connect(self.reqthread.deleteLater)
        self.reqworker.data.connect(self.results)
        self.reqthread.start()
    
    
    #### get status from laser class
    def update_status(self,data_retrieved):
        self.laser.current = data_retrieved[0]
        self.laser.power = data_retrieved[1]
        self.laser.TEC_temperature = data_retrieved[2]
        self.laser.TEC_curent=data_retrieved[3]
        self.laser.status=data_retrieved[4]
        self.update_laser_values()
    
    #### 
    def update_laser_values(self):
        self.current_display.display(self.laser.current)
        self.power_display.display(self.laser.power)
        self.currTEC_display.display(self.laser.TEC_current)
        self.temp_display.display(self.laser.TEC_temperature)
        self.status=self.laser.status
        self.button_state()
        
    def button_state(self):
        if self.status=='Disabled' or self.status=='Ready':
            self.turn_on_button.setText('Start Laser')
             
        elif self.status=='Active':
            self.turn_on_button.setText('Disable')

        elif self.status=='Hibernating':
            self.turn_on_button.setText('Restart')
    
        elif self.status=='Configuring':
            self.turn_on_button.setText('Disable')
    
    def shutdown(self):
        self.q.put_nowait('shutdown')
    
    def laser_clicked(self):
        if self.status=='Disabled':
            self.q.put_nowait('enable')
            
        elif self.status=='Active':
            self.q.put_nowait('disable')
            
        elif self.status=='Hibernating':
            self.q.put_nowait('reset')
        
        elif self.status=='Ready':
            self.q.put_nowait('enable')
    
        elif self.status=='Configuring':
            self.q.put_nowait('disable')
    
    def results(self,data):
        print(data)
        
    def current_changed(self):
        print(self.currentv.text())
        self.laser.set_current(int(self.currentv.text()))
    
    def temp_changed(self):
        print(self.temperaturev.text())
        self.laser.set_TEC_temperature(int(self.temperaturev.text())*10)
        
    
    ##############################################laser thread
        
    def temp_alarm(self):
        if (float(self.laser.TEC_temperature)<24.9 or (self.laser.TEC_temperature)>25.1):
            msg = QMessageBox()
            msg.setIcon(QMessageBox.Critical)
            msg.setText("Error")
            msg.setInformativeText('TEC Temperature is unstable, shutting down the laser')
            msg.setWindowTitle("Error")
            msg.exec_() 
    
    def closeEvent(self, event):
        self.worker.stop()
        self.thread.quit()
        self.reqworker.stop()
        self.reqthread.quit()
        #self.q.join()
        print('Thread Closed')
        self.laser.close_port()
        event.accept()
    
        
    
qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
#     file = QFile(":/dark/stylesheet.qss")
#     file.open(QFile.ReadOnly | QFile.Text)
#     stream = QTextStream(file)
#     qapp.setStyleSheet(stream.readAll())
    
if __name__ == "__main__": 
    #start the widget
    ui = laser_widget()
    #show the widget
    ui.show()
    #start the events loop
    qapp.exec_()


Serial port closed


# Data Acquisition

In [1]:
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from nidaqmx.stream_readers import AnalogMultiChannelReader
from nidaqmx.constants import AcquisitionType, WAIT_INFINITELY
import numpy as np
import os
from DAQ import DAQ

import sys

class acquisition_widget(QWidget):
    """
    This is my widget.
    """
    def __init__(self, parent=None):
        super().__init__()
        
        
        self.global_layout = QHBoxLayout(self)
        
        layout1=QVBoxLayout(self)
        
        #file name
        self.filelabel=QLabel(self)
        self.filelabel.setText('File Name')
        layout1.addWidget(self.filelabel)
        
        self.filename=QLineEdit(self)
        self.filename.setText('')
        layout1.addWidget(self.filename)
        
        #sample rate
        self.samplelabel=QLabel(self)
        self.samplelabel.setText('Sample Rate (kHz)')
        layout1.addWidget(self.samplelabel)
        
        self.samplerate=QLineEdit(self)
        self.samplerate.setText('')
        layout1.addWidget(self.samplerate)
        
        self.samplerate.returnPressed.connect(self.update_sample_rate)
        
        #acquisition time
        self.acqlabel=QLabel(self)
        self.acqlabel.setText('Acquisition Time (s)')
        layout1.addWidget(self.acqlabel)
        
        self.acqtime=QLineEdit(self)
        self.acqtime.setText('')
        layout1.addWidget(self.acqtime)
        
        #button
        
        self.recording_button=QToolButton(self)
        self.recording_button.setText('Start Recording')
        self.recording_button.clicked.connect(self.thread)
        
        layout1.addWidget(self.recording_button)
        
        
        self.global_layout.addLayout(layout1)
        
        #self.setFixedWidth(300)
        #self.setFixedHeight(300)
    def thread(self):
        self.thread=QThread()
        self.worker=Worker_DAQ(int(self.acqtime.text()),int(self.samplerate.text()),self.filename.text())
        self.worker.moveToThread(self.thread)
        self.thread.started.connect(self.worker.start_recording)
        self.worker.finished.connect(self.thread.quit)
        self.worker.finished.connect(self.worker.deleteLater)
        self.thread.finished.connect(self.thread.deleteLater)
        self.thread.start()
        
    def update_sample_rate(self):
        if float(self.samplerate.text())<=10:
            self.samplerate.setText(str(self.samplerate.text()))
        else:
            msg = QMessageBox()
            msg.setIcon(QMessageBox.Critical)
            msg.setText("Error")
            msg.setInformativeText('The sample rate is too high')
            msg.setWindowTitle("Error")
            msg.exec_()
            self.samplerate.setText('0')
    
    def closeEvent(self, event):
        self.worker.stop()
        self.thread.exit()
        event.accept()
        
class Worker_DAQ(QWidget):
    finished=pyqtSignal()
    data=pyqtSignal()
    
    def __init__(self,acq,sr,file):
        super(Worker_DAQ,self).__init__()
        self.daq=DAQ()
        self.acq=acq
        self.sr=sr*1000
        self.file=file
    
    def start_recording(self):
        print('baby')
        n_points=self.sr*self.acq
        self.daq.task.timing.cfg_samp_clk_timing(rate=self.sr,
                                sample_mode=AcquisitionType.CONTINUOUS,
                                samps_per_chan=n_points)
        
        self.daq.task.start()
        reader = AnalogMultiChannelReader(self.daq.task.in_stream)
        self.data_out = np.empty(shape=(3, n_points))
        reader.read_many_sample(data = self.data_out,number_of_samples_per_channel=n_points,timeout=WAIT_INFINITELY)
        
        self.daq.task.stop()
        
        print('Acquisition Sucessful')
        
        self.file_save(self.data_out,self.file,self.sr)
        
    def file_save(self,data,filename,sr):
        default_dir =""
        default_filename = os.path.join(default_dir, filename)
        filepath, _ = QFileDialog.getSaveFileName(self, "Save file", default_filename, "(*.dat)")
        
        if filepath:
            np.savetxt(filepath, np.column_stack(data),header='Samples per channel: ' + str(sr) +'\nX\tY\tSUM')
        self.finished.emit()
        
qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
    
if __name__ == "__main__": 
    #start the widget
    ui = acquisition_widget()
    #show the widget
    ui.show()
    #start the events loop
    qapp.exec_()

DaqError: Device cannot be accessed.  Possible causes:

Device is no longer present in the system.
Device is not powered.
Device is powered, but was temporarily without power.
Device is damaged.

Ensure the device is properly connected and powered.  Turn the computer off and on again.  If you suspect that the device is damaged, contact National Instruments at ni.com/support.
Device Specified: OTKB

Task Name: _unnamedTask<0>

Status Code: -201003

AttributeError: 'acquisition_widget' object has no attribute 'worker'

# Piezos

In [5]:
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from pyqtgraph import PlotWidget, plot
import pyqtgraph as pg
import numpy as np
from piezos import KPA, list_devices

import sys

class piezo_widget(QWidget):
    
    def __init__(self, parent=None):
        super().__init__(parent)
        
        self.kpa=KPA(list_devices()[3])
        
        self.kpa.connect()
        
        self.global_layout = QHBoxLayout(self)
        self.layout1=QVBoxLayout(self)
        self.layout2=QVBoxLayout(self)
        
        
        sizePolicy=QSizePolicy(QSizePolicy.Expanding,QSizePolicy.Minimum)
        # displacement controls 
        self.dislabel=QLabel(self)
        self.dislabel.setText('Enter Displacement')
        self.dislabel.setSizePolicy(sizePolicy)
        self.layout1.addWidget(self.dislabel)
        
        #x axis
        self.xlabel=QLabel(self)
        self.xlabel.setText('X-axis(nm)')
        self.xlabel.setSizePolicy(sizePolicy)
        self.layout1.addWidget(self.xlabel)
        
        self.x_axis=QLineEdit(self)
        self.x_axis.setText('')
        self.x_axis.setSizePolicy(sizePolicy)
        self.layout1.addWidget(self.x_axis)
        
        #y axis
        self.ylabel=QLabel(self)
        self.ylabel.setText('Y-axis(nm)')
        self.ylabel.setSizePolicy(sizePolicy)
        self.layout1.addWidget(self.ylabel)
        
        self.y_axis=QLineEdit(self)
        self.y_axis.setText('')
        self.y_axis.setSizePolicy(sizePolicy)
        self.layout1.addWidget(self.y_axis)
        
        #z axis
        self.zlabel=QLabel(self)
        self.zlabel.setText('Z-axis(nm)')
        self.zlabel.setSizePolicy(sizePolicy)
        self.layout1.addWidget(self.zlabel)
        
        self.z_axis=QLineEdit(self)
        self.z_axis.setText('')
        self.z_axis.setSizePolicy(sizePolicy)
        self.layout1.addWidget(self.z_axis)
        
        #buttons
        
        self.turn_button=QToolButton(self)
        self.turn_button.setText('Turn On')
        self.turn_button.setSizePolicy(sizePolicy)
        self.layout2.addWidget(self.turn_button)
        
        self.reset_button=QToolButton(self)
        self.reset_button.setText('Reset')
        self.reset_button.setSizePolicy(sizePolicy)
        self.layout2.addWidget(self.reset_button)
        
        self.zero_button=QToolButton(self)
        self.zero_button.setText('Zero')
        self.zero_button.setSizePolicy(sizePolicy)
        self.layout2.addWidget(self.zero_button)
        ################################ Plot ###################################################
        
        self.layout3=QVBoxLayout(self)
        
        self.graphWidget = pg.PlotWidget()
        
        self.layout3.addWidget(self.graphWidget)

        self.x = [ self.kpa.get_position()[0] ] 
        self.y = [ self.kpa.get_position()[1] ]
        
        self.graphWidget.setBackground('w')
        
        self.graphWidget.showGrid(x = True, y = True, alpha = 1.0)
        
        self.graphWidget.setXRange(-0.008, 0.008, padding=0)
        self.graphWidget.setYRange(-0.008, 0.008, padding=0)

        pen = pg.mkPen(color=(255, 0, 0))
        self.data_line =  self.graphWidget.plot(self.x, self.y, pen=pen, symbol='o')
        
        self.global_layout.addLayout(self.layout1)
        self.global_layout.addLayout(self.layout2)
        self.global_layout.addLayout(self.layout3)
        
        #self.setFixedWidth(300)
        #self.setFixedHeight(200)
        self.thread()
    
    def thread(self):
        self.thread=QThread()
        self.worker=Worker_kpa(self.kpa)
        self.worker.moveToThread(self.thread)
        self.thread.started.connect(self.worker.update)
        self.worker.finished.connect(self.thread.quit)
        self.worker.finished.connect(self.worker.deleteLater)
        self.thread.finished.connect(self.thread.deleteLater)
        self.worker.data.connect(self.update_plot_data)
        self.thread.start()
    
    def update_plot_data(self,data_retrieved):
        self.x = [ data_retrieved[0] ]
        self.y = [ data_retrieved[1] ]
        self.data_line.setData(self.x, self.y)  # Update the data
        
    def closeEvent(self, event):
        self.kpa.close()
        self.worker.stop()
        self.thread.quit()
        print('Closed, bby!')
        event.accept()
        

class Worker_kpa(QObject):
    finished=pyqtSignal()
    data=pyqtSignal(list)
    
    def __init__(self,kpa):
        super(Worker_kpa,self).__init__()
        self.kpa=kpa
        self.flag=True
        
    def update(self):
        while self.flag:
            #QThread.sleep(1)
            data=self.kpa.get_position()
            self.data.emit([ data[0],data[1] ])
        self.finished.emit()
    
    def start(self):
        self.flag=True
        
    def stop(self):
        self.flag=False
        #self.kpa.close()


qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
    
if __name__ == "__main__": 
    #start the widget
    ui = piezo_widget()
    #show the widget
    ui.show()
    #start the events loop
    qapp.exec_()

Thorlabs.MotionControl.DeviceManagerCLI.SingleDeviceInfo
agora bby
Closed, bby!


# Live Mode

In [6]:
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *

import sys

class live_mode(QWidget):
    """
    This is my widget.
    """
    def __init__(self):
        super().__init__()
        
        self.global_layout = QGridLayout(self)
        
        #laser container
        containerl=QGroupBox("Laser Controls")
        layout=QHBoxLayout(containerl)
        
        self.laser=laser_widget(self)
        layout.addWidget(self.laser)
        containerl.setLayout(layout)
        
        self.global_layout.addWidget(containerl,0,0)
        
        
        #Data Acquisition
        containeracq=QGroupBox("Data Parameters")
        layout1=QHBoxLayout(containeracq)
        
        self.data=acquisition_widget(self)
        layout1.addWidget(self.data)
        containeracq.setLayout(layout1)
        
        self.global_layout.addWidget(containeracq,1,1)
        
        #camera
        containercam=QGroupBox("Camera")
        layout=QHBoxLayout(containercam)
        
        self.camera=camera_widget()
        layout.addWidget(self.camera)
        containercam.setLayout(layout)
        
        self.global_layout.addWidget(containercam,0,1)
        
        #piezo controls
        containerpiezo=QGroupBox("Piezo Controls")
        layout=QHBoxLayout(containerpiezo)
        
        self.piezo=piezo_widget(self)
        layout.addWidget(self.piezo)
        containerpiezo.setLayout(layout)
        
        self.global_layout.addWidget(containerpiezo,1,0)
        
    def closeEvent(self, event):
        self.laser.closeEvent(event)
        self.camera.closeEvent(event)
        self.piezo.closeEvent(event)
        event.accept()
        
qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
    
if __name__ == "__main__": 
    #start the widget
    ui = live_mode()
    #show the widget
    ui.show()
    #start the events loop
    qapp.exec_()

Thorlabs.MotionControl.DeviceManagerCLI.SingleDeviceInfo
agora bby

Serial port closed


PortNotOpenError: Attempting to use a port that is not open

Closed, bby!


sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1c1fdecf7c0>


# Main Window

In [7]:
#interface 2.0
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
import breeze_resources
import sys

class interface(QWidget):
    """
    This is my widget.
    """
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle("Ferdinand")
        
        #layout principal
        self.global_layout = QHBoxLayout(self)
        
        #initiate tabs
        self.live=live_mode()
        
        self.tabs = QTabWidget()
        self.tab1 = QWidget()
        self.tab2 = QWidget()
        self.tab3 = QWidget()
        self.tab4 = QWidget()
        
        self.tabs.addTab(self.tab1,"Live Mode")
        self.tabs.addTab(self.tab2,"Calibration Mode")
        self.tabs.addTab(self.tab3,"Classification Mode")
        self.tabs.addTab(self.tab4,"Settings")        
        
        #tab1- Live mode
        
        self.tab1.layout = QHBoxLayout(self)
        self.tab1.layout.addWidget(self.live)
        self.tab1.setLayout(self.tab1.layout)
        self.global_layout.addWidget(self.tabs)
        self.setLayout(self.global_layout)
        
        #tab2- Calibration Mode
        
    def closeEvent(self, event):
        close = QMessageBox.question(self,"Quit","Are you sure want to stop process?",QMessageBox.Yes | QMessageBox.No)
        if close == QMessageBox.Yes:
            self.live.closeEvent(event)
            event.accept()
        else:
            event.ignore()
        
###################################################################
qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
    file = QFile(":/dark/stylesheet.qss")
    file.open(QFile.ReadOnly | QFile.Text)
    stream = QTextStream(file)
    qapp.setStyleSheet(stream.readAll())
    
if __name__ == "__main__": 
    #start the widget
    ui = interface()
    #show the widget
    ui.show()
    #start the event loop
    qapp.exec_()

Thorlabs.MotionControl.DeviceManagerCLI.SingleDeviceInfo
agora bby
baby
Acquisition Sucessful

Serial port closed
Closed, bby!
